In [1]:
import pandas as pd
import numpy as np

In [2]:
solar_pw = pd.read_csv('./solar_power_generation.csv')

In [3]:
X = solar_pw[['DAILY_YIELD','TOTAL_YIELD','AMBIENT_TEMPERATURE','MODULE_TEMPERATURE','IRRADIATION','DC_POWER']]
y = solar_pw['AC_POWER']

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.2,random_state=21)

In [5]:
from sklearn.linear_model import ElasticNet
model = ElasticNet(random_state=0)
model.fit(X_train,y_train)

ElasticNet(random_state=0)

In [6]:
model.score(X_test,y_test)

0.9999937122356861

In [7]:
prediction= model.predict(X_test)

In [8]:
prediction

array([3.24212737e-01, 1.85999599e+02, 9.70434883e+02, ...,
       2.00685572e-01, 2.04932484e-01, 3.61150814e+02])

In [9]:
actual_predicted = pd.DataFrame({'Actual' : y_test , 'Predicted' : prediction})

In [10]:
actual_predicted

,Actual,Predicted
39704,0.000000,0.324213
16578,186.325000,185.999599
38467,971.014286,970.434883
19471,0.000000,0.340822
21836,0.000000,0.308717
...,...,...
10459,0.000000,0.295123
27264,0.000000,0.220551
19030,0.000000,0.200686
12827,0.000000,0.204932


In [11]:
actual_predicted['Error'] = actual_predicted['Actual'] - actual_predicted['Predicted']

In [12]:
actual_predicted.head()

,Actual,Predicted,Error
39704,0.000000,0.324213,-0.324213
16578,186.325000,185.999599,0.325401
38467,971.014286,970.434883,0.579403
19471,0.000000,0.340822,-0.340822
21836,0.000000,0.308717,-0.308717


In [ ]:
import sklearn
from skops import hub_utils
import pickle

# let's save the model
model_path = "./solar.pkl"
local_repo = "my-solar-model"
with open(model_path, mode="bw") as f:
    pickle.dump(model, file=f)

# we will now initialize a local repository
hub_utils.init(
    model=model_path, 
    requirements=[f"scikit-learn={sklearn.__version__}"], 
    dst=local_repo,
    task="tabular-regression",
    data=X_test,
)

In [ ]:
from skops import card

# create the card 
model_card = card.Card(model, metadata=card.metadata_from_config(local_repo))

limitations = "This model is not ready to be used in production."
model_description = "This is a LinearRegression model trained on Solar Power Generation Data."
model_card_authors = "ayyuce demirbas"
get_started_code = "import pickle \nwith open(dtc_pkl_filename, 'rb') as file: \n    clf = pickle.load(file)"
citation_bibtex = "bibtex\n@inproceedings{...,year={2022}}"

# we can add the information using add
model_card.add(
    citation_bibtex=citation_bibtex,
    get_started_code=get_started_code,
    model_card_authors=model_card_authors,
    limitations=limitations,
    model_description=model_description,
)

# we can set the metadata part directly
model_card.metadata.license = "gnu"

In [ ]:
model_card.add(eval_method="The model is evaluated using test split, on accuracy.")
model_card.add_metrics(accuracy= 100 * model.score(X_test,y_test))

In [ ]:
model_card.save(local_repo + "/README.md")

In [ ]:
token = 'your_awesome_token'

In [ ]:
repo_id = "ayyuce/my_solar_model"
hub_utils.push(
    repo_id=repo_id,
    source=local_repo,
    token=token,
    commit_message="pushing files to the repo from the example!",
    create_remote=True,
)